# **Querying with SPARQL**


In [ ]:
!pip install rdflib

In [ ]:
from rdflib import Graph, URIRef
from rdflib.namespace import RDFS, SKOS

In [ ]:
g = Graph()
g.parse('https://www.wikidata.org/wiki/Special:EntityData/Q2831.ttl')

<Graph identifier=Nfea12afe5d7d4ce9a935a5160d277d09 (<class 'rdflib.graph.Graph'>)>

In [ ]:
len(g)

102252

In [ ]:
MJ = URIRef('http://www.wikidata.org/entity/Q2831')
for label in g.objects(MJ, SKOS.altLabel):
    print(label.value, label.language)

In [ ]:
labels = g.preferredLabel(MJ, labelProperties=(SKOS.altLabel,))
for prop, label in labels:
    print(label.value, label.language)

In [ ]:
qres = g.query('''
    SELECT ?label
    WHERE {
        wd:Q2831 skos:altLabel ?label .
    }
''')
for label, *_ in qres:
    print(label.value, label.language)

In [ ]:
qres = g.query('''
    SELECT DISTINCT ?label
    WHERE {
        wd:Q2831 rdfs:label | skos:prefLabel | skos:altLabel ?label .
        FILTER (lang(?label) = 'en')
    }
    ORDER BY ?label
''')
for label, *_ in qres:
    print(label)

In [ ]:
qres = g.query('''
    SELECT ?label ?description
    WHERE {
        wd:Q2831 wdt:P166 ?award .
        ?award rdfs:label ?label .
        FILTER (lang(?label) = 'en')
        OPTIONAL {
            ?award schema:description ?description
            FILTER (lang(?description) = 'en')
        }
    }
''')

for label, description in qres:
    print(f'Award: {label:<55} Description: {description}')

In [ ]:
%timeit -n 1000 list(g.objects(MJ, RDFS.label))

1000 loops, best of 3: 744 µs per loop


In [ ]:
%timeit -n 1000 g.preferredLabel(MJ, labelProperties=(RDFS.label,))

1000 loops, best of 3: 803 µs per loop


In [ ]:
%timeit -n 1000 g.query('SELECT ?label WHERE {wd:Q2831 rdfs:label ?label}')

1000 loops, best of 3: 3.52 ms per loop
